# Análisis de Supervivencia en el Titanic

En este cuaderno se realiza un Análisis Exploratorio de Datos (EDA) sobre el famoso conjunto de datos del Titanic. El objetivo principal es responder a la pregunta de negocio: **¿Qué factores están relacionados con que los pasajeros tengan una mayor o menor probabilidad de sobrevivir?** Además, se entrenará un modelo de regresión logística para clasificar a los pasajeros según la variable `Survived`.

## Carga de librerías y datos

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Cargar datos
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

df.head()

Mostramos información general del *DataFrame*, así como la cantidad de valores faltantes por columna.

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

## Estadísticas básicas

In [ ]:
df.describe(include='all')

Analicemos la proporción de supervivientes frente a no supervivientes.

In [ ]:
survival_counts = df['Survived'].value_counts()
survival_rates = df['Survived'].value_counts(normalize=True) * 100
print('Conteo de supervivencia:')
print(survival_counts)
print('
Porcentaje de supervivencia:')
print(survival_rates)


### Supervivencia por sexo

In [ ]:
sns.barplot(data=df, x='Sex', y='Survived')
plt.title('Tasa de supervivencia por sexo')
plt.show()

Se observa que las mujeres presentaron una tasa de supervivencia muy superior a la de los hombres.

### Supervivencia por clase de pasajero

In [ ]:
sns.barplot(data=df, x='Pclass', y='Survived')
plt.title('Tasa de supervivencia por clase')
plt.show()

La clase también parece influir: los pasajeros de primera clase sobrevivieron en mayor proporción que los de tercera.

### Distribución de edades

In [ ]:
sns.histplot(data=df, x='Age', hue='Survived', multiple='stack')
plt.title('Distribución de edades por supervivencia')
plt.show()

Aunque existen valores faltantes en la edad, la gráfica sugiere que los pasajeros más jóvenes tenían una ligera ventaja.

## Preparación de datos para el modelo

In [ ]:
df_model = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].copy()

# Imputación de valores faltantes
df_model['Age'].fillna(df_model['Age'].median(), inplace=True)
df_model['Embarked'].fillna(df_model['Embarked'].mode()[0], inplace=True)

X = df_model.drop('Survived', axis=1)
y = df_model['Survived']

# Definición de columnas categóricas y numéricas
cat_cols = ['Sex', 'Embarked']
num_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

# Preprocesamiento: OneHotEncoder para variables categóricas
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(drop='first'), cat_cols)],
    remainder='passthrough'
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


## Entrenamiento del modelo

In [ ]:
model = Pipeline(steps=[('preprocess', preprocessor),
                        ('classifier', LogisticRegression(max_iter=200))])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('Exactitud:', accuracy_score(y_test, y_pred))
print('
Reporte de clasificación:
', classification_report(y_test, y_pred))

Los resultados muestran la precisión y las métricas de clasificación. Un valor de exactitud por encima del 70% es habitual con este tipo de modelo y sin excesivo preprocesamiento.

## Conclusiones

* **Sexo**: ser mujer incrementó notablemente la probabilidad de sobrevivir.
* **Clase del billete**: los pasajeros de primera clase tuvieron mayores tasas de supervivencia que los de clases inferiores.
* **Edad**: los pasajeros más jóvenes presentaron cierta ventaja, aunque el efecto no es tan marcado como el del sexo o la clase.

El modelo de regresión logística confirma que estas variables aportan información valiosa para predecir la supervivencia. Sin embargo, no es un modelo perfecto y existen otros factores (por ejemplo, el número de familiares a bordo o la tarifa pagada) que también influyen en menor medida.